In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

import math

In [28]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [29]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [20]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_weights['item_id'])
    item_weights = np.array(items_weights['weight'])
    recs = np.random.choice(items, size=n, replace=False, p=item_weights)
    
    return recs.tolist()

def log_shift1(x):
    return math.log(x+1)

In [21]:
%%time

# your_code
# items_weights = data.groupby('item_id')['quantity'].sum().reset_index()
# items_weights['weight'] = items_weights['quantity'] / items_weights['quantity'].sum()
items_weights = data.groupby('item_id')['sales_value'].sum().reset_index()
items_weights['log_sum'] = items_weights['sales_value'].map(lambda b: log_shift1(b))
items_weights['weight'] = items_weights['log_sum'] / items_weights['log_sum'].sum()

data_test['weighted_random_recommendation'] = data_test['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
data_test.head(3)

Wall time: 3min


<timed exec>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,weighted_random_recommendation
2277416,338,41260573635,636,840173,1,1.99,369,0.0,112,92,0.0,0.0,"[1056427, 849707, 834777, 822101, 279994]"
2277417,338,41260573635,636,1037348,1,0.89,369,-0.3,112,92,0.0,0.0,"[9671465, 10204433, 1514813, 873130, 7414651]"
2277418,338,41260573635,636,5592737,2,1.58,369,-0.2,112,92,0.0,0.0,"[1000325, 12300991, 887270, 999172, 12172388]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [67]:
result = pd.read_csv('./predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [68]:
def str_to_array(x):
    x = str.replace(x, '[', '')
    x = str.replace(x, ']', '')
    x = np.fromstring(x, dtype=int, count=-1, sep=',')
    
    return x

def precision_at_k(recommended_list, bought_list, k=5):
    
    if type(recommended_list) == str:
        recommended_list = str_to_array(recommended_list)
    elif type(recommended_list) == list:
        recommended_list = np.array(recommended_list)

    if type(bought_list) == str:
        bought_list = str_to_array(bought_list)
    elif type(bought_list) == list:
        bought_list = np.array(bought_list)
        
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    precision = flags.sum() / len(recommended_list)
    
    return precision


In [73]:
# your_code

result['prec_random_rec'] = result.apply(lambda x: precision_at_k(x.random_recommendation, x.actual, 5), axis=1)
result['prec_popular_rec'] = result.apply(lambda x: precision_at_k(x.popular_recommendation, x.actual, 5), axis=1)
result['prec_itemitem'] = result.apply(lambda x: precision_at_k(x.itemitem, x.actual, 5), axis=1)
result['prec_cosine'] = result.apply(lambda x: precision_at_k(x.cosine, x.actual, 5), axis=1)
result['prec_tfidf'] = result.apply(lambda x: precision_at_k(x.tfidf, x.actual, 5), axis=1)
result['prec_own_purchases'] = result.apply(lambda x: precision_at_k(x.own_purchases, x.actual, 5), axis=1)


<ipython-input-68-417a34877ddd>:4: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  x = np.fromstring(x, dtype=int, count=-1, sep=',')


In [74]:
result.describe()

,user_id,prec_random_rec,prec_popular_rec,prec_itemitem,prec_cosine,prec_tfidf,prec_own_purchases
count,2042.000000,2042.0,2042.000000,2042.000000,2042.000000,2042.000000,2042.000000
mean,1257.930950,0.0,0.009892,0.001763,0.001469,0.001469,0.009109
std,718.052041,0.0,0.043376,0.018699,0.017083,0.017083,0.041709
min,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,648.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1260.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1879.750000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
max,2500.000000,0.0,0.200000,0.200000,0.200000,0.200000,0.200000


#### Лучше остальных показали себя популярные и собственные рекомендации. Случайные вообще ничего не угадали

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [75]:
# your_code